# Shortterm Load Prediction using Transfer Learning

In [56]:
# Import python libraries
#
import pandas as pd
import numpy as np
import importlib
from tqdm import tqdm
import pickle
import holidays
import pytz
from joblib import Parallel, delayed
import torch
import plotly.graph_objects as go

# Imports own modules.
# All imports are done relative to the root of the project.
# Therefore please add the project root to your environment variables.
#
import sys
sys.path.append('../')
import Model_xLSTM as model
import data.weather_data as weather_data
import data.demandprofiles_readout as demandprofiles
import data.standardprofiles_readout as standardprofiles
import LstmAdapter as LstmAdapter
import visualization as visualization


In [57]:
importlib.reload(weather_data)
importlib.reload(demandprofiles)
importlib.reload(standardprofiles)
importlib.reload(LstmAdapter)

# Readout the preprocessed power profiles
#
household_loads_train = pd.read_pickle('../data/loadprofiles_1h_train.pkl')
household_loads_test = pd.read_pickle('../data/loadprofiles_1h_test.pkl')
household_loads_train['Aggregated_Power'] = household_loads_train.sum(axis=1)
powerProfile = -1.0 * household_loads_train['Aggregated_Power']

# Readout the weather data
#
startDate = powerProfile.index[0].to_pydatetime().replace(tzinfo=None)
endDate = powerProfile.index[-1].to_pydatetime().replace(tzinfo=None)
weather_measurements = weather_data.WeatherMeasurements()
weatherData = weather_measurements.get_data(
            startDate = startDate, 
            endDate = endDate,
            lat = 51.4817,      # Location:
            lon = 7.2165,       # Bochum Germany,
            alt = 102,          # Meteostat weatherstation   
            sample_periode = 'hourly', 
            tz = 'Europe/Vienna',
            )
selected_columns = ['temp', 'prcp', 'wspd', 'tsun']     # select weather data
weatherData = weatherData.loc[:, selected_columns]

# Load the public holiday calendar
public_holidays_dict = holidays.CountryHoliday('DE', years=range(2013, 2014))
public_holidays_timestamps = [pd.Timestamp(date, tzinfo=pytz.timezone('Europe/Vienna')) for date in public_holidays_dict.keys()]

# Bring the aggregated characteristic power profiles to the format needed by the model
#
lstmAdapter = LstmAdapter.LstmAdapter(public_holidays_timestamps, train_size = 263, dev_size = 0, 
                                            add_tda_features=False, addLaggedPower=True, 
                                            shuffle_data=False, seed=0)
X_model, Y_model = lstmAdapter.transformData(powerProfile, weatherData)

# Readout the standard power profiles (for pretraining)
# and bring it to the format needed by the model
#
demandProfiles_Readout = standardprofiles.StandardProfiles_Readout()
standardprofiles_scaled = demandProfiles_Readout.get_scaled_standardprofiles(powerProfile)
lstmAdapter_pretrain = LstmAdapter.LstmAdapter(public_holidays_timestamps, train_size = 263, dev_size = 0, 
                                            add_tda_features=False, addLaggedPower=True, 
                                            shuffle_data=False, seed=0)
X_model_pretrain, Y_model_pretrain = lstmAdapter_pretrain.transformData(standardprofiles_scaled, None)


In [58]:
importlib.reload(model)
importlib.reload(visualization)

# Pretrain the power profile model
#
pretrain_model = False
if pretrain_model == True:

    optimizer = model.Optimizer(maxEpochs=300, set_learning_rates=[0.015, 0.005, 0.003, 0.002, 0.001, 0.001, 0.001])
    model_pretraining = model.Model(optimizer, reg_strength = 0.00)
    model_pretraining.compile(metrics=[])
    history_pretraining = model_pretraining.model.fit(x=X_model_pretrain['train'], y=Y_model_pretrain['train'], 
                        validation_data=(X_model_pretrain['dev'], Y_model_pretrain['dev']),
                        batch_size=optimizer.miniBatchSize,
                        epochs=optimizer.maxEpochs,
                        shuffle=True, verbose='auto',
                        callbacks=[
                            optimizer.get_lr_callback(), 
                            #optimizer.get_early_stopping_callbacks()
                            ]
                        )
    
    # Store the results
    #
    model_pretraining.save_weights('lstm_pretrain_weights.h5')
    with open('lstm_pretrain_results.pkl', 'wb') as file:
        pickle.dump((history_pretraining, model_pretraining.model), file)

    # Evaluate the pretrained model
    #
    # plotlyApp = visualization.PlotlyApp(X_model_pretrain, Y_model_pretrain, model_pretraining, lstmAdapter_pretrain)
    # plotlyApp.run(myport=8055)
    

In [81]:
# Train with moving horizon
#

# Reload the model module
importlib.reload(model)

# Define the training days. Train not every day-of-year, but
# only the first 14 days and weekly afterwards.
training_days = list(range(1, X_model['all'].shape[0], 31))
# training_days = [300]

def train_on_day(i, use_transfer_learning, X, Y):
    X_past = X['all'][:i, :, :]
    Y_past = Y['all'][:i, :, :]    

    num_of_features = X_past.shape[2]
    myModel = model.Model(num_of_features)
    if use_transfer_learning == True:
        myModel.load_weights('lstm_pretrain_weights.h5')
    train_history = myModel.train_model(X_past, Y_past, verbose=1)

    return myModel, train_history, i

def train_on_day_wrapper(args):
    return train_on_day(*args)

def start_training(use_transfer_learning):
    num_workers = 1
    results = Parallel(n_jobs=num_workers)(
        delayed(train_on_day)(i, use_transfer_learning, X_model, Y_model) for i in training_days
    )

    models_list, train_histories_list, day_indices = zip(*results)
    train_histories = {i: history for i, history in zip(day_indices, train_histories_list)}
    models = {i: model for i, model in zip(day_indices, models_list)}

    return train_histories, models

def evaluate_results(models):
    evaluations = {}
    predictions = {}

    for i in tqdm(range(1, X_model['all'].shape[0]), desc="Evaluation and Prediction Progress"):
        
        # Get the last trained model to a given day
        last_training_day = max(filter(lambda key: key <= i, models.keys()))
        last_available_model = models[last_training_day]

        # Evaluate and predict
        X_future = X_model['all'][i, np.newaxis, :, :]
        Y_future = Y_model['all'][i, np.newaxis, :, :]
        evaluations[i] = last_available_model.evaluate(X_future, Y_future)
        predictions[i] = last_available_model.predict(torch.from_numpy(X_future).float())

    return evaluations, predictions


In [82]:
train_histories, models = start_training(use_transfer_learning=False)


Epoch 1/100 - Loss: 11.8260 - LR = 0.01
Epoch 2/100 - Loss: 11.5849 - LR = 0.01
Epoch 3/100 - Loss: 11.4457 - LR = 0.01
Epoch 4/100 - Loss: 11.2793 - LR = 0.01
Epoch 5/100 - Loss: 11.1256 - LR = 0.01
Epoch 6/100 - Loss: 10.9202 - LR = 0.01
Epoch 7/100 - Loss: 10.5252 - LR = 0.01
Epoch 8/100 - Loss: 9.6527 - LR = 0.01
Epoch 9/100 - Loss: 8.7303 - LR = 0.01
Epoch 10/100 - Loss: 7.9204 - LR = 0.01
Epoch 11/100 - Loss: 7.1530 - LR = 0.01
Epoch 12/100 - Loss: 6.3913 - LR = 0.01
Epoch 13/100 - Loss: 5.6470 - LR = 0.01
Epoch 14/100 - Loss: 4.8845 - LR = 0.01
Epoch 15/100 - Loss: 4.0939 - LR = 0.01
Epoch 16/100 - Loss: 3.3478 - LR = 0.01
Epoch 17/100 - Loss: 2.6763 - LR = 0.01
Epoch 18/100 - Loss: 2.1406 - LR = 0.01
Epoch 19/100 - Loss: 1.8164 - LR = 0.01
Epoch 20/100 - Loss: 1.7681 - LR = 0.01
Epoch 21/100 - Loss: 1.9977 - LR = 0.01
Epoch 22/100 - Loss: 2.3706 - LR = 0.01
Epoch 23/100 - Loss: 2.6363 - LR = 0.01
Epoch 24/100 - Loss: 2.5966 - LR = 0.01
Epoch 25/100 - Loss: 2.2410 - LR = 0.01
Ep

Epoch 1/100 - Loss: 10.3067 - LR = 0.01
Epoch 2/100 - Loss: 8.9592 - LR = 0.01
Epoch 3/100 - Loss: 7.7970 - LR = 0.01
Epoch 4/100 - Loss: 6.7836 - LR = 0.01
Epoch 5/100 - Loss: 5.8515 - LR = 0.01
Epoch 6/100 - Loss: 4.9116 - LR = 0.01
Epoch 7/100 - Loss: 3.9875 - LR = 0.01
Epoch 8/100 - Loss: 3.1120 - LR = 0.01
Epoch 9/100 - Loss: 2.2987 - LR = 0.01
Epoch 10/100 - Loss: 1.6646 - LR = 0.01
Epoch 11/100 - Loss: 1.3319 - LR = 0.01
Epoch 12/100 - Loss: 1.3831 - LR = 0.01
Epoch 13/100 - Loss: 1.7521 - LR = 0.01
Epoch 14/100 - Loss: 2.1268 - LR = 0.01
Epoch 15/100 - Loss: 2.2093 - LR = 0.01
Epoch 16/100 - Loss: 1.9589 - LR = 0.01
Epoch 17/100 - Loss: 1.5458 - LR = 0.01
Epoch 18/100 - Loss: 1.3115 - LR = 0.01
Epoch 19/100 - Loss: 1.4590 - LR = 0.01
Epoch 20/100 - Loss: 1.3352 - LR = 0.01
Epoch 21/100 - Loss: 1.3486 - LR = 0.01
Epoch 22/100 - Loss: 1.3435 - LR = 0.01
Epoch 23/100 - Loss: 1.3257 - LR = 0.01
Epoch 24/100 - Loss: 1.3102 - LR = 0.01
Epoch 25/100 - Loss: 1.3054 - LR = 0.01
Epoch 26

Epoch 1/100 - Loss: 11.0389 - LR = 0.01
Epoch 2/100 - Loss: 9.1685 - LR = 0.01
Epoch 3/100 - Loss: 8.0115 - LR = 0.01
Epoch 4/100 - Loss: 7.0132 - LR = 0.01
Epoch 5/100 - Loss: 6.0512 - LR = 0.01
Epoch 6/100 - Loss: 5.2207 - LR = 0.01
Epoch 7/100 - Loss: 4.4102 - LR = 0.01
Epoch 8/100 - Loss: 3.5903 - LR = 0.01
Epoch 9/100 - Loss: 2.7954 - LR = 0.01
Epoch 10/100 - Loss: 2.0897 - LR = 0.01
Epoch 11/100 - Loss: 1.5500 - LR = 0.01
Epoch 12/100 - Loss: 1.2593 - LR = 0.01
Epoch 13/100 - Loss: 1.2768 - LR = 0.01
Epoch 14/100 - Loss: 1.5690 - LR = 0.01
Epoch 15/100 - Loss: 1.9440 - LR = 0.01
Epoch 16/100 - Loss: 2.1585 - LR = 0.01
Epoch 17/100 - Loss: 2.1048 - LR = 0.01
Epoch 18/100 - Loss: 1.8320 - LR = 0.01
Epoch 19/100 - Loss: 1.4861 - LR = 0.01
Epoch 20/100 - Loss: 1.2530 - LR = 0.01
Epoch 21/100 - Loss: 1.2736 - LR = 0.01
Epoch 22/100 - Loss: 1.4429 - LR = 0.01
Epoch 23/100 - Loss: 1.5071 - LR = 0.01
Epoch 24/100 - Loss: 1.4562 - LR = 0.01
Epoch 25/100 - Loss: 1.3660 - LR = 0.01
Epoch 26

Epoch 1/100 - Loss: 8.7440 - LR = 0.01
Epoch 2/100 - Loss: 8.1686 - LR = 0.01
Epoch 3/100 - Loss: 7.8058 - LR = 0.01
Epoch 4/100 - Loss: 7.3273 - LR = 0.01
Epoch 5/100 - Loss: 6.8249 - LR = 0.01
Epoch 6/100 - Loss: 6.3085 - LR = 0.01
Epoch 7/100 - Loss: 5.7710 - LR = 0.01
Epoch 8/100 - Loss: 5.2095 - LR = 0.01
Epoch 9/100 - Loss: 4.6507 - LR = 0.01
Epoch 10/100 - Loss: 4.0528 - LR = 0.01
Epoch 11/100 - Loss: 3.3585 - LR = 0.01
Epoch 12/100 - Loss: 2.6690 - LR = 0.01
Epoch 13/100 - Loss: 2.0495 - LR = 0.01
Epoch 14/100 - Loss: 1.5381 - LR = 0.01
Epoch 15/100 - Loss: 1.2027 - LR = 0.01
Epoch 16/100 - Loss: 1.1262 - LR = 0.01
Epoch 17/100 - Loss: 1.3331 - LR = 0.01
Epoch 18/100 - Loss: 1.6654 - LR = 0.01
Epoch 19/100 - Loss: 1.8446 - LR = 0.01
Epoch 20/100 - Loss: 1.7411 - LR = 0.01
Epoch 21/100 - Loss: 1.4300 - LR = 0.01
Epoch 22/100 - Loss: 1.1442 - LR = 0.01
Epoch 23/100 - Loss: 1.2444 - LR = 0.01
Epoch 24/100 - Loss: 1.2995 - LR = 0.01
Epoch 25/100 - Loss: 1.2162 - LR = 0.01
Epoch 26/

Epoch 1/100 - Loss: 8.6426 - LR = 0.01
Epoch 2/100 - Loss: 7.6484 - LR = 0.01
Epoch 3/100 - Loss: 6.6225 - LR = 0.01
Epoch 4/100 - Loss: 5.8341 - LR = 0.01
Epoch 5/100 - Loss: 5.1163 - LR = 0.01
Epoch 6/100 - Loss: 4.4003 - LR = 0.01
Epoch 7/100 - Loss: 3.6884 - LR = 0.01
Epoch 8/100 - Loss: 2.9819 - LR = 0.01
Epoch 9/100 - Loss: 2.3129 - LR = 0.01
Epoch 10/100 - Loss: 1.7246 - LR = 0.01
Epoch 11/100 - Loss: 1.2831 - LR = 0.01
Epoch 12/100 - Loss: 1.0603 - LR = 0.01
Epoch 13/100 - Loss: 1.1143 - LR = 0.01
Epoch 14/100 - Loss: 1.3963 - LR = 0.01
Epoch 15/100 - Loss: 1.7124 - LR = 0.01
Epoch 16/100 - Loss: 1.8524 - LR = 0.01
Epoch 17/100 - Loss: 1.7524 - LR = 0.01
Epoch 18/100 - Loss: 1.4811 - LR = 0.01
Epoch 19/100 - Loss: 1.1664 - LR = 0.01
Epoch 20/100 - Loss: 1.0538 - LR = 0.01
Epoch 21/100 - Loss: 1.1776 - LR = 0.01
Epoch 22/100 - Loss: 1.1579 - LR = 0.01
Epoch 23/100 - Loss: 1.1124 - LR = 0.01
Epoch 24/100 - Loss: 1.0800 - LR = 0.01
Epoch 25/100 - Loss: 1.0610 - LR = 0.01
Epoch 26/

...

Epoch 1/100 - Loss: 7.2469 - LR = 0.01
Epoch 2/100 - Loss: 6.2081 - LR = 0.01
Epoch 3/100 - Loss: 5.0024 - LR = 0.01
Epoch 4/100 - Loss: 3.7977 - LR = 0.01
Epoch 5/100 - Loss: 2.7041 - LR = 0.01
Epoch 6/100 - Loss: 1.8543 - LR = 0.01
Epoch 7/100 - Loss: 1.2178 - LR = 0.01
Epoch 8/100 - Loss: 0.8967 - LR = 0.01
Epoch 9/100 - Loss: 0.9639 - LR = 0.01
Epoch 10/100 - Loss: 1.2923 - LR = 0.01
Epoch 11/100 - Loss: 1.5143 - LR = 0.01
Epoch 12/100 - Loss: 1.3965 - LR = 0.01
Epoch 13/100 - Loss: 1.0626 - LR = 0.01
Epoch 14/100 - Loss: 0.8790 - LR = 0.01
Epoch 15/100 - Loss: 1.0807 - LR = 0.01
Epoch 16/100 - Loss: 0.9576 - LR = 0.01
Epoch 17/100 - Loss: 0.8908 - LR = 0.01
Epoch 18/100 - Loss: 0.8760 - LR = 0.01
Epoch 19/100 - Loss: 0.8812 - LR = 0.01
Epoch 20/100 - Loss: 0.8886 - LR = 0.01
Epoch 21/100 - Loss: 0.8900 - LR = 0.01
Epoch 22/100 - Loss: 0.8854 - LR = 0.01
Epoch 23/100 - Loss: 0.8792 - LR = 0.01
Epoch 24/100 - Loss: 0.8758 - LR = 0.01
Epoch 25/100 - Loss: 0.8767 - LR = 0.01
Epoch 26/

Epoch 1/100 - Loss: 6.6109 - LR = 0.01
Epoch 2/100 - Loss: 5.6394 - LR = 0.01
Epoch 3/100 - Loss: 4.5164 - LR = 0.01
Epoch 4/100 - Loss: 3.3471 - LR = 0.01
Epoch 5/100 - Loss: 2.3904 - LR = 0.01
Epoch 6/100 - Loss: 1.7147 - LR = 0.01
Epoch 7/100 - Loss: 1.2156 - LR = 0.01
Epoch 8/100 - Loss: 0.9193 - LR = 0.01
Epoch 9/100 - Loss: 0.8712 - LR = 0.01
Epoch 10/100 - Loss: 1.0461 - LR = 0.01
Epoch 11/100 - Loss: 1.2496 - LR = 0.01
Epoch 12/100 - Loss: 1.2715 - LR = 0.01
Epoch 13/100 - Loss: 1.1024 - LR = 0.01
Epoch 14/100 - Loss: 0.8901 - LR = 0.01
Epoch 15/100 - Loss: 0.9487 - LR = 0.01
Epoch 16/100 - Loss: 0.9590 - LR = 0.01
Epoch 17/100 - Loss: 0.8923 - LR = 0.01
Epoch 18/100 - Loss: 0.8623 - LR = 0.01
Epoch 19/100 - Loss: 0.8580 - LR = 0.01
Epoch 20/100 - Loss: 0.8635 - LR = 0.01
Epoch 21/100 - Loss: 0.8675 - LR = 0.01
Epoch 22/100 - Loss: 0.8674 - LR = 0.01
Epoch 23/100 - Loss: 0.8636 - LR = 0.01
Epoch 24/100 - Loss: 0.8592 - LR = 0.01
Epoch 25/100 - Loss: 0.8573 - LR = 0.01
Epoch 26/

Epoch 1/100 - Loss: 7.5836 - LR = 0.01
Epoch 2/100 - Loss: 6.9357 - LR = 0.01
Epoch 3/100 - Loss: 6.3820 - LR = 0.01
Epoch 4/100 - Loss: 5.7731 - LR = 0.01
Epoch 5/100 - Loss: 5.1089 - LR = 0.01
Epoch 6/100 - Loss: 4.5030 - LR = 0.01
Epoch 7/100 - Loss: 3.9395 - LR = 0.01
Epoch 8/100 - Loss: 3.3802 - LR = 0.01
Epoch 9/100 - Loss: 2.8215 - LR = 0.01
Epoch 10/100 - Loss: 2.2721 - LR = 0.01
Epoch 11/100 - Loss: 1.7689 - LR = 0.01
Epoch 12/100 - Loss: 1.3423 - LR = 0.01
Epoch 13/100 - Loss: 1.0336 - LR = 0.01
Epoch 14/100 - Loss: 0.8995 - LR = 0.01
Epoch 15/100 - Loss: 0.9724 - LR = 0.01
Epoch 16/100 - Loss: 1.1908 - LR = 0.01
Epoch 17/100 - Loss: 1.3827 - LR = 0.01
Epoch 18/100 - Loss: 1.4065 - LR = 0.01
Epoch 19/100 - Loss: 1.2519 - LR = 0.01
Epoch 20/100 - Loss: 1.0195 - LR = 0.01
Epoch 21/100 - Loss: 0.8859 - LR = 0.01
Epoch 22/100 - Loss: 0.9768 - LR = 0.01
Epoch 23/100 - Loss: 1.0339 - LR = 0.01
Epoch 24/100 - Loss: 0.9931 - LR = 0.01
Epoch 25/100 - Loss: 0.9307 - LR = 0.01
Epoch 26/

Epoch 1/100 - Loss: 4.9709 - LR = 0.01
Epoch 2/100 - Loss: 4.3448 - LR = 0.01
Epoch 3/100 - Loss: 3.7851 - LR = 0.01
Epoch 4/100 - Loss: 3.2227 - LR = 0.01
Epoch 5/100 - Loss: 2.6436 - LR = 0.01
Epoch 6/100 - Loss: 2.0803 - LR = 0.01
Epoch 7/100 - Loss: 1.5896 - LR = 0.01
Epoch 8/100 - Loss: 1.1983 - LR = 0.01
Epoch 9/100 - Loss: 0.9531 - LR = 0.01
Epoch 10/100 - Loss: 0.9278 - LR = 0.01
Epoch 11/100 - Loss: 1.1221 - LR = 0.01
Epoch 12/100 - Loss: 1.3278 - LR = 0.01
Epoch 13/100 - Loss: 1.3372 - LR = 0.01
Epoch 14/100 - Loss: 1.1477 - LR = 0.01
Epoch 15/100 - Loss: 0.9326 - LR = 0.01
Epoch 16/100 - Loss: 0.9821 - LR = 0.01
Epoch 17/100 - Loss: 1.0126 - LR = 0.01
Epoch 18/100 - Loss: 0.9621 - LR = 0.01
Epoch 19/100 - Loss: 0.9245 - LR = 0.01
Epoch 20/100 - Loss: 0.9096 - LR = 0.01
Epoch 21/100 - Loss: 0.9084 - LR = 0.01
Epoch 22/100 - Loss: 0.9124 - LR = 0.01
Epoch 23/100 - Loss: 0.9161 - LR = 0.01
Epoch 24/100 - Loss: 0.9168 - LR = 0.01
Epoch 25/100 - Loss: 0.9146 - LR = 0.01
Epoch 26/

Epoch 1/100 - Loss: 5.7327 - LR = 0.01
Epoch 2/100 - Loss: 5.1048 - LR = 0.01
Epoch 3/100 - Loss: 4.5540 - LR = 0.01
Epoch 4/100 - Loss: 3.9731 - LR = 0.01
Epoch 5/100 - Loss: 3.4407 - LR = 0.01
Epoch 6/100 - Loss: 2.9536 - LR = 0.01
Epoch 7/100 - Loss: 2.4592 - LR = 0.01
Epoch 8/100 - Loss: 1.9821 - LR = 0.01
Epoch 9/100 - Loss: 1.5537 - LR = 0.01
Epoch 10/100 - Loss: 1.2204 - LR = 0.01
Epoch 11/100 - Loss: 1.0258 - LR = 0.01
Epoch 12/100 - Loss: 1.0203 - LR = 0.01
Epoch 13/100 - Loss: 1.1951 - LR = 0.01
Epoch 14/100 - Loss: 1.3776 - LR = 0.01
Epoch 15/100 - Loss: 1.3996 - LR = 0.01
Epoch 16/100 - Loss: 1.2473 - LR = 0.01
Epoch 17/100 - Loss: 1.0484 - LR = 0.01
Epoch 18/100 - Loss: 1.0196 - LR = 0.01
Epoch 19/100 - Loss: 1.1242 - LR = 0.01
Epoch 20/100 - Loss: 1.0944 - LR = 0.01
Epoch 21/100 - Loss: 1.0399 - LR = 0.01
Epoch 22/100 - Loss: 1.0061 - LR = 0.01
Epoch 23/100 - Loss: 0.9973 - LR = 0.01
Epoch 24/100 - Loss: 1.0022 - LR = 0.01
Epoch 25/100 - Loss: 1.0107 - LR = 0.01
Epoch 26/

In [99]:
importlib.reload(model)

evaluations, predictions = evaluate_results(models)


Evaluation and Prediction Progress:   0%|          | 0/342 [00:00<?, ?it/s]

Evaluation and Prediction Progress: 100%|██████████| 342/342 [00:18<00:00, 18.09it/s]


In [100]:
evaluations

{1: (1.8321099281311035, 1.8321099281311035),
 2: (1.304588794708252, 1.304588794708252),
 3: (1.2797602415084839, 1.2797602415084839),
 4: (1.3393349647521973, 1.3393349647521973),
 5: (1.1763032674789429, 1.1763032674789429),
 6: (1.15060555934906, 1.15060555934906),
 7: (1.6309655904769897, 1.6309655904769897),
 8: (1.859804391860962, 1.859804391860962),
 9: (1.350881814956665, 1.350881814956665),
 10: (1.2883440256118774, 1.2883440256118774),
 11: (1.3113495111465454, 1.3113495111465454),
 12: (1.3853710889816284, 1.3853710889816284),
 13: (1.3314929008483887, 1.3314929008483887),
 14: (1.5640177726745605, 1.5640177726745605),
 15: (2.237881660461426, 2.237881660461426),
 16: (1.357262134552002, 1.357262134552002),
 17: (1.2731077671051025, 1.2731077671051025),
 18: (1.2636250257492065, 1.2636250257492065),
 19: (1.4915651082992554, 1.4915651082992554),
 20: (1.304419755935669, 1.304419755935669),
 21: (1.5455949306488037, 1.5455949306488037),
 22: (1.780385136604309, 1.78038513660

In [101]:
# Compare the pretrained and non-pretrained results
#

def get_final_metrics(history):
    return {key: value['loss'][-1][0] for key, value in history.items()}

def get_final_val_metrics(evaluations):
    return evaluations.copy()

def create_trace(x_data, y_data, name):
    return go.Scatter(x=list(x_data.keys()), y=list(y_data.values()), mode='lines', name=name)

def create_layout(title, x_title, y_title):
    return go.Layout(
        title=title,
        xaxis=dict(title=x_title),
        yaxis=dict(title=y_title),
        legend=dict(x=1, y=1, traceorder='normal', orientation='v')  # Adjusted legend position
    )

def create_and_show_figure(traces, layout):
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

# Get final metrics
losses = get_final_metrics(train_histories)
val_losses = get_final_val_metrics(evaluations)

# Create figures and traces
#
trace_train_loss = create_trace(losses, losses, 'train_loss_with_pretraining')
trace_val_loss = create_trace(val_losses, val_losses, 'val_loss_with_pretraining')
layout_train_loss = create_layout('Training Loss', 'Days', 'MSE')
layout_val_loss = create_layout('Validation Loss', 'Days', 'MSE')
fig_train_loss = go.Figure(data=[trace_train_loss], layout=layout_train_loss)
fig_val_loss = go.Figure(data=[trace_val_loss], layout=layout_val_loss)
create_and_show_figure([trace_train_loss], layout_train_loss)
create_and_show_figure([trace_val_loss], layout_val_loss)


In [102]:
importlib.reload(visualization)

# Evaluate the finetuned models
#
plotlyApp = visualization.PlotlyApp(X_model, Y_model, None, lstmAdapter, predictions, )#Y_model_pretrain, lstmAdapter_pretrain)
plotlyApp.run(myport=8050)



In [103]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

def plot_loss(fig, row_num, histories, first_epoch, last_epoch, skip_first_days):
    for run, history in histories.items():
        if run < skip_first_days:
            print("Day skipped!")
            continue
        loss = np.array(history['loss'][first_epoch:last_epoch])
        fig.add_trace(go.Scatter(x=np.arange(len(loss)), y=loss, mode='lines', name=f'Day {run}', showlegend=False), row=row_num, col=1)

first_epoch = 0
last_epoch = -1
first_day = 0

# Create Training
fig = sp.make_subplots(rows=3, cols=1, 
                       subplot_titles=['Training Loss during Pretraining', 
                                       'Training Loss with Transfer Learning',
                                       'Training Loss without Transfer Learning'], 
                        shared_xaxes=True)
# Set the same scale on the y-axis for all subplots
fig.update_yaxes(title_text="Mean Squared Error", row=1, col=1, range=[0, 0.15])     # type='log'
fig.update_yaxes(title_text="Mean Squared Error", row=2, col=1, range=[0, 0.15])
fig.update_yaxes(title_text="Mean Squared Error", row=3, col=1, range=[0, 0.15])
fig.update_xaxes(title_text="Training Epochs", row=3, col=1)
fig.update_layout(height=800, width=800)

# Plot training loss for each history object
plot_loss(fig, 2, train_histories, first_epoch, last_epoch, first_day)

fig.write_image('training_loss.png', scale=4)

fig.show()
